In [1]:
#csv파일이랑 audio폴더 test 양만큼 개줄여서 디버그하며 할 것

import os
import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import pandas as pd
import csv

import scipy
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.svm import SVC

In [2]:
filename = np.array(pd.read_csv('data/SC_file_name.csv'))
answer = np.array(pd.read_csv('data/SC_answer.csv'))

audio_train_files = os.listdir('./audio_train/')
audio_test_files = os.listdir('./audio_test/')
train=pd.read_csv('./file_name.csv')

In [3]:
#print(audio_train_files)
#print(audio_test_files)

In [4]:
def get_mfcc(name, path):
    data, _ = librosa.core.load(path+name) #엑셀에 .wav까지 붙여 저장해야됨 안그러면 path+name+'.wav' 이렇게해야
    print(data,_)
    #data, _ = librosa.load(path)
    try:
        #ft1 = librosa.feature.mfcc(data, sr = SAMPLE_RATE, n_mfcc=30)
        ft1 = librosa.feature.mfcc(data, n_mfcc=30) #이게핵심 Sample_rate를 어떻게 조절하느냐
        ft2 = librosa.feature.zero_crossing_rate(data)[0] # n_mfcc 값을 어떻게 조절하느냐에 따라 Feature개수가 달리 뽑힘
        ft3 = librosa.feature.spectral_rolloff(data)[0]
        ft4 = librosa.feature.spectral_centroid(data)[0]
        ft5 = librosa.feature.spectral_contrast(data)[0]
        ft6 = librosa.feature.spectral_bandwidth(data)[0]
        ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.median(ft1, axis = 1), np.min(ft1, axis = 1)))
        ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
        ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
        ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
        ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
        ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
        return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))
    except:
        print('bad file')
        return pd.Series([0]*210)

In [5]:
train_data = pd.DataFrame()
train_data['fname'] = train['fname']
test_data=pd.DataFrame()
test_data['fname']=audio_test_files

In [6]:
#print(train_data)
print(test_data)


                        fname
0  130_1p3_Al_mc_AKGC417L.wav
1  130_1p3_Ar_mc_AKGC417L.wav
2  130_1p3_Ll_mc_AKGC417L.wav
3  130_1p3_Lr_mc_AKGC417L.wav
4  130_1p3_Pl_mc_AKGC417L.wav
5  130_1p3_Pr_mc_AKGC417L.wav
6  130_1p3_Tc_mc_AKGC417L.wav
7  130_1p4_Al_mc_AKGC417L.wav


In [7]:
train_data=train_data['fname'].progress_apply(get_mfcc,path='./audio_train/')
print('done loading train mfcc')
test_data = test_data['fname'].progress_apply(get_mfcc, path='./audio_test/')
print('done loading test mfcc')



  0%|                                                                                           | 0/99 [00:00<?, ?it/s]

[0.06973912 0.10266001 0.09116188 ... 0.08305359 0.08018959 0.08612607] 22050


  2%|█▋                                                                                 | 2/99 [00:06<05:03,  3.13s/it]

[-0.02710845 -0.03993477 -0.03530764 ... -0.04230448 -0.0414599
 -0.04527015] 22050


  3%|██▌                                                                                | 3/99 [00:08<04:36,  2.88s/it]

...

 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [00:09<00:01,  1.38s/it]

[ 0.          0.          0.         ... -0.09460735 -0.09186393
 -0.10033422] 22050


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:10<00:00,  1.40s/it]

[0.         0.         0.         ... 0.06824436 0.06823804 0.07497056] 22050


done loading test mfcc


In [8]:
train_data['fname'] = train['fname']
train_data['label'] = train['label']

test_data['fname'] = audio_test_files
test_data['label'] = np.zeros((len(audio_test_files)))

In [9]:
#print(train_data)
print(test_data)

            0           1          2          3          4          5  \
0 -374.568476  121.653345  68.564955  35.947216  30.422790  31.903702   
1 -397.516973   97.320405  71.506663  47.443845  33.989614  28.778871   
2 -309.153916  108.590946  49.665790  39.516736  30.495293  28.091219   
3 -270.489556  161.700804  49.739469  17.809513  13.739956  18.044573   
4 -266.609507  158.940668  69.044580  38.555959  28.130570  21.142545   
5 -286.585566  149.852568  74.347056  30.249630  16.780876  14.380454   
6 -243.604437  144.016071   9.083950  25.720157  23.012074  19.829217   
7 -333.424073  134.835047  62.990353  28.851085  28.611614  31.429581   

           6          7          8          9  ...          202       203  \
0  27.481430  19.513747  14.341624  13.400741  ...    18.338631  7.468611   
1  24.618514  18.687007  13.254794  10.825098  ...    18.039230  7.656408   
2  23.433613  16.137411  14.933775  15.932992  ...    20.245296  5.962564   
3  16.730022  12.005675  10.303434

In [21]:
X = train_data.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(train_data.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in train_data.label.values])

#print(X)
#y는 train 데이터들이 라벨링한 것
print(y)

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]


In [11]:
X_test = test_data.drop(['label', 'fname'], axis=1)
X_test = X_test.values

In [24]:
print(X.shape)
print(X_test.shape)
#test의 라벨링은 어디서?

(99, 210)
(8, 210)


In [13]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
#이놈이 뭘까

In [14]:
#pca = PCA(n_components=65).fit(X_scaled)
#65땜에 오류뜸
pca = PCA(n_components=65).fit(X_scaled)
#이 값 고쳐야
X_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(sum(pca.explained_variance_ratio_)) 

0.9927819852312187


In [28]:
# Fit an SVM model
X_train, X_val, y_train, y_val = train_test_split(X_pca, y, test_size = 0.2, random_state = 42, shuffle = True)
#test_size와 random_state 확인

#train 관련
#print(X_train)
#print(y_train) - train 라벨링

#train 내에서 쪼개는 것으로 추측됨 test 0.2가 100개중 20개로 맞네
#test를 따로 만드는게 아니라 train 내에서 일부를 test에 사용하여 정확도 측정
print(X_val)
print(y_val)

clf = SVC(kernel = 'rbf', probability=True)

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

[[ 4.88100404 -3.43397136 -3.29845388 ... -0.61609378  0.23458665
   0.97816446]
 [ 0.26160425  5.18237066 -2.43019333 ... -0.24010789 -0.35883319
  -0.09569657]
 [ 0.91343711 -2.02287154  2.09438045 ... -0.59792007 -1.20865358
  -0.2661965 ]
 ...
 [-6.1306718  -4.3883038  -6.27430465 ...  0.18317393  0.20533995
   0.42573345]
 [ 2.40666755  3.78935566  9.21096189 ... -0.21731497  0.16284491
  -0.09165545]
 [ 1.16133406  8.34994398  7.72972024 ...  0.04968694  0.01360946
  -0.70441906]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.95


In [16]:
C_grid = [0.001, 0.01, 0.1, 1, 10]
gamma_grid = [0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': C_grid, 'gamma' : gamma_grid}

#grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
#이부분 수정해야
#데이터 갯수 확 느니까 바로먹음
grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
grid.fit(X_train, y_train)

# Find the best model
print(grid.best_score_)

print(grid.best_params_)

print(grid.best_estimator_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.9746835443037974
{'C': 0.001, 'gamma': 0.001}
SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [17]:

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

0.95


In [18]:
# Fit the entire training sets
clf.fit(X_pca, y)


SVC(C=4, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
'''
#나중에 위로 올릴 것
def convert_to_labels(preds, i2c, k=3):
    ans = []
    ids = []
    for p in preds:
        idx = np.argsort(p)[::-1]
        ids.append([i for i in idx[:k]])
        ans.append(' '.join([i2c[i] for i in idx[:k]]))

    return ans, ids

#test 폴더 submission 만들기 - 오류
submission = pd.read_csv('./sample_submission.csv')

str_preds, _ = convert_to_labels(clf.predict_proba(X_test_pca), i2c, k=3)

# Write to outputs
subm = pd.DataFrame()
subm['fname'] = audio_test_files
subm['label'] = str_preds
subm.to_csv('submission.csv', index=False)
'''

"\n#나중에 위로 올릴 것\ndef convert_to_labels(preds, i2c, k=3):\n    ans = []\n    ids = []\n    for p in preds:\n        idx = np.argsort(p)[::-1]\n        ids.append([i for i in idx[:k]])\n        ans.append(' '.join([i2c[i] for i in idx[:k]]))\n\n    return ans, ids\n\n#test 폴더 submission 만들기 - 오류\nsubmission = pd.read_csv('./sample_submission.csv')\n\nstr_preds, _ = convert_to_labels(clf.predict_proba(X_test_pca), i2c, k=3)\n\n# Write to outputs\nsubm = pd.DataFrame()\nsubm['fname'] = audio_test_files\nsubm['label'] = str_preds\nsubm.to_csv('submission.csv', index=False)\n"